# Notebook pro trénink s destilací nad datasetem CIFAR10
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR10, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

In [1]:
%pip install transformers[torch] huggingface_hub datasets evaluate torchvision optuna

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import knihoven a definice metod

In [1]:
from transformers import Trainer, TrainingArguments, MobileNetV2Config, MobileNetV2ForImageClassification, EarlyStoppingCallback
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
import torch.nn as nn
import numpy as np
import evaluate
import random
import pickle
import optuna
import torch
import math
import os 

Resetování náhodného seedu pro replikovatelnost výsledků.
Zřejmě je možné části odebrat.

TODO: Odebrat zbytečná nastavení.

In [2]:
def reset_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

Nový wrapper, který pracuje přímo se soubory staženého a upraveného datasetu CIFAR10.
Využití načtení pomocí metody jako dříve není možné kvůli jiné checksum. 

Zároveň se již dotahují logity přímo z datasetu.

In [3]:
class CustomCIFAR10(Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None):
        self.root = root
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        self.data = []
        self.targets = []
        self.logits = []
        
        if self.train:
             for i in range(1, 6):
                 data_file = os.path.join(self.root, 'cifar-10-batches-py', f'data_batch_{i}')
                 with open(data_file, 'rb') as fo:
                     dict = pickle.load(fo, encoding='bytes')
                     self.data.append(dict[b'data'])
                     self.targets.extend(dict[b'labels'])
                     self.logits.extend(dict[b'logits'])  
        else:
            data_file = os.path.join(self.root, 'cifar-10-batches-py', 'test_batch')
            with open(data_file, 'rb') as fo:
                dict = pickle.load(fo, encoding='bytes')
                self.data.append(dict[b'data'])
                self.targets.extend(dict[b'labels'])
                self.logits.extend(dict[b'logits'])  

        self.data = np.concatenate(self.data, axis=0)
        self.targets = np.array(self.targets)
        self.logits = np.array(self.logits)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index].reshape(3, 32, 32).transpose(1, 2, 0)
        label = self.targets[index]
        logit = self.logits[index]
        
        image = Image.fromarray(image.astype('uint8'), 'RGB')
        logit = torch.tensor(logit, dtype=torch.float)
        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            target = self.target_transform(target)
            
        return {
            'pixel_values': image,
            'labels': label,
            'logits': logit
        }


Definice accuracy metriky pro trénování modelu.

In [17]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    pred, labels = eval_pred
    predictions = np.argmax(pred, axis=1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='macro', zero_division = 0)
    recall = recall_metric.compute(predictions=predictions, references=labels, average='macro', zero_division = 0)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }

Trénovací argumenty pro trainer. 

In [18]:
class Custom_training_args(TrainingArguments):
    def __init__(self, lambda_param, temperature, *args, **kwargs):
        super().__init__(*args, **kwargs)    
        self.lambda_param = lambda_param
        self.temperature = temperature

In [19]:
def get_training_args(output_dir:str, logging_dir:str, remove_unused_columns:bool):
    return (
        Custom_training_args(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=5e-5, #Defaultní hodnota 
        per_device_train_batch_size=128,
        per_device_eval_batch_size=128,
        num_train_epochs=20,
        weight_decay=0.01,
        seed = 42,  #Defaultní hodnota 
        metric_for_best_model="f1",
        fp16=True, 
        logging_dir=logging_dir,
        remove_unused_columns=remove_unused_columns,
        lambda_param = 0.5, 
        temperature = 5
    ))

Náhodně inicializovaný MobileNetV2.

In [20]:
def get_random_init_mobilenet():
    reset_seed(42)
    student_config = MobileNetV2Config()
    student_config.num_labels = 10
    return MobileNetV2ForImageClassification(student_config)

In [21]:
reset_seed(42)

In [22]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
print(os.getcwd())
test = CustomCIFAR10(root='./data/10-logits', train=False, transform=transform)
train = CustomCIFAR10(root='./data/10-logits', train=True, transform=transform)

/home/jovyan


### Standardní trénink náhodně inicializovaného modelu. 

In [24]:
training_args = get_training_args("./results/cifar10-random", './logs/cifar10-random', True)

In [25]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "adam_beta1" : trial.suggest_float("adam_beta1", 0.9, 0.99, step=0.01)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [26]:
#Nápočet epoch na steps
min_r = math.ceil(50000/128)*3
max_r = math.ceil(50000/128)*20

In [27]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [28]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    model_init=get_random_init_mobilenet
  )
  

In [16]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test",
    n_trials=150
)

[I 2025-01-04 21:20:33,926] A new study created in memory with name: Test


Trial 0 with params: {'learning_rate': 1.0253509690168497e-05, 'weight_decay': 0.01, 'adam_beta1': 0.97}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.324600,1.952858,0.279100,0.279871,0.279100,0.259486
2,1.959300,1.712043,0.350800,0.347448,0.350800,0.342523
3,1.810900,1.617426,0.392200,0.389229,0.392200,0.385845


[I 2025-01-04 21:27:26,053] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 4.128205343826226e-05, 'weight_decay': 0.001, 'adam_beta1': 0.91}


...

Trial 117 with params: {'learning_rate': 0.00016769203705822665, 'weight_decay': 0.008, 'adam_beta1': 0.92}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.655600,1.142616,0.582600,0.590841,0.582600,0.584549
2,1.033300,0.819238,0.712200,0.717916,0.712200,0.712892
3,0.737400,0.644444,0.778800,0.784087,0.778800,0.779574
4,0.564400,0.590818,0.798300,0.800921,0.798300,0.796929
5,0.440600,0.571379,0.800400,0.810360,0.800400,0.802439
6,0.335600,0.602637,0.801900,0.827367,0.801900,0.804206


[I 2025-01-07 02:45:25,489] Trial 117 pruned. 


Trial 118 with params: {'learning_rate': 0.00045039424742782536, 'weight_decay': 0.0, 'adam_beta1': 0.9500000000000001}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.470900,1.018190,0.628200,0.628457,0.628200,0.626227
2,0.927500,0.786155,0.718300,0.724879,0.718300,0.716573
3,0.702600,0.661289,0.767300,0.777702,0.767300,0.765997
4,0.568900,0.574138,0.804000,0.802679,0.804000,0.800934
5,0.477800,0.554265,0.810200,0.818739,0.810200,0.811700
6,0.397400,0.523596,0.826200,0.836739,0.826200,0.826352
7,0.327000,0.552846,0.826300,0.836050,0.826300,0.825359
8,0.261700,0.591155,0.820700,0.826989,0.820700,0.818889
9,0.200400,0.509233,0.840400,0.849822,0.840400,0.842428
10,0.151900,0.499016,0.855100,0.857505,0.855100,0.855394


In [ ]:
print(best_trial.hyperparameters)

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

In [17]:
class ImageDistilTrainer(Trainer):
    def __init__(self, model_init, *args, **kwargs):
        self.model_init = model_init
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
       
        super().__init__(model_init=model_init, *args, **kwargs)
        
        self.student = self.model_init()
        self.temperature = self.args.temperature
        self.lambda_param = self.args.lambda_param

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.student.to(device)


    def compute_loss(self, model, inputs, return_outputs=False):
        logits = inputs.pop("logits")

        student_output = model(**inputs)
        self.lambda_param = self.args.lambda_param
        self.temperature = self.args.temperature
        
        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        student_target_loss = student_output.loss

        loss = ((1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss)
        return (loss, student_output) if return_outputs else loss

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [18]:
reset_seed(42)

In [19]:
training_args = get_training_args("./results/cifar10-random-KD", './logs/cifar10-random-KD', False)

In [20]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }

In [21]:
trainer = ImageDistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    model_init=get_random_init_mobilenet,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [22]:
pruner = optuna.pruners.HyperbandPruner(min_resource=1, reduction_factor=4)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [23]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    n_trials=80
)

[I 2025-01-02 10:31:58,319] A new study created in memory with name: no-name-c949043e-bbe5-4784-8e94-3711ffa24f5e


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.826100,1.794448,0.356200,0.363023,0.356200,0.332477
2,1.663900,1.638281,0.429000,0.451496,0.429000,0.430032
3,1.506000,1.536666,0.490100,0.496502,0.490100,0.486227
4,1.396100,1.514397,0.516500,0.521185,0.516500,0.506952
5,1.311700,1.508902,0.527800,0.575985,0.527800,0.532914
6,1.262200,1.367924,0.586400,0.578568,0.586400,0.576121
7,1.198600,1.423449,0.576800,0.601245,0.576800,0.572559
8,1.125700,1.765431,0.509900,0.563286,0.509900,0.494695
9,1.058500,1.310685,0.626400,0.646086,0.626400,0.620627
10,1.029100,1.226045,0.662000,0.674270,0.662000,0.660745


[I 2025-01-02 11:14:36,692] Trial 0 finished with value: 0.6493830756403727 and parameters: {'learning_rate': 1.0253509690168497e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.001, 'lambda_param': 0.1, 'temperature': 2.0}. Best is trial 0 with value: 0.6493830756403727.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.650800,1.645658,0.641600,0.651808,0.641600,0.630255
2,0.467800,1.807167,0.725200,0.751262,0.725200,0.725268
3,0.372000,1.774964,0.796000,0.797620,0.796000,0.793375
4,0.311600,1.782879,0.820700,0.821734,0.820700,0.819950
5,0.266400,1.785201,0.801300,0.828130,0.801300,0.804786
6,0.232700,1.714340,0.823700,0.833181,0.823700,0.820774
7,0.202900,1.730780,0.816000,0.830533,0.816000,0.814600
8,0.177800,1.671794,0.767400,0.791711,0.767400,0.761982
9,0.154400,1.754976,0.850000,0.855467,0.850000,0.849485
10,0.136000,1.758484,0.853400,0.860455,0.853400,0.854400


[I 2025-01-02 12:06:48,211] Trial 1 finished with value: 0.8775070142695549 and parameters: {'learning_rate': 0.00021766241123453658, 'per_device_train_batch_size': 32, 'weight_decay': 0.01, 'lambda_param': 0.9, 'temperature': 3.0}. Best is trial 1 with value: 0.8775070142695549.


...

[I 2025-01-02 19:20:27,459] Trial 77 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.594300,1.438909,0.170700,0.148205,0.170700,0.118925


[I 2025-01-02 19:23:30,635] Trial 78 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.609300,1.698447,0.615400,0.631447,0.615400,0.606751


[I 2025-01-02 19:26:47,257] Trial 79 pruned. 
